# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
import_path = "../WeatherPy/Output/cities.csv"

weather_data_df = pd.read_csv(import_path, dtype="object", encoding="utf-8")
weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,County,Date
0,0,avarua,-21.21,-159.78,80.6,74.0,61.0,3.36,CK,2020-10-19 23:33:08.243463
1,1,klaksvik,62.23,-6.59,37.4,69.0,16.0,5.82,FO,2020-10-19 23:33:08.335434
2,2,sioux lookout,50.1,-91.92,28.4,79.0,40.0,3.36,CA,2020-10-19 23:33:08.396412
3,3,albany,42.6,-73.97,57.0,83.0,100.0,7.45,US,2020-10-19 23:33:08.442417
4,4,cabedelo,-6.98,-34.83,73.4,88.0,20.0,5.82,BR,2020-10-19 23:33:08.508377


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_data_df[['Lat', 'Lng']].astype(float).dropna()
humidity_weight = weather_data_df['Humidity'].astype(float).dropna()

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_conditions_df = weather_data_df.loc[(weather_data_df['Max Temp'] < '80') &
                                            (weather_data_df['Max Temp'] > '70') &
                                            (weather_data_df['Wind Speed'] < '10') &
                                            (weather_data_df['Cloudiness'] < '50')]

weather_conditions_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,County,Date
96,96,bilma,18.69,12.92,75.42,19.0,0.0,1.5,NE,2020-10-19 23:33:13.877093
145,145,santa isabel do rio negro,-0.41,-65.02,73.72,95.0,100.0,1.63,BR,2020-10-19 23:33:16.987484
207,207,mandleshwar,22.18,75.67,75.2,83.0,31.0,1.77,IN,2020-10-19 23:33:20.880812
353,353,fengxian,34.7,116.59,71.01,52.0,100.0,1.01,CN,2020-10-19 23:33:30.182207
524,524,port macquarie,-31.43,152.92,71.01,56.0,29.0,1.99,AU,2020-10-19 23:33:41.396418
531,531,nakanojo,36.58,138.85,71.01,45.0,16.0,1.01,JP,2020-10-19 23:33:41.959239
540,540,riberalta,-10.98,-66.1,72.9,88.0,100.0,1.66,BO,2020-10-19 23:33:42.495068


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_conditions_df
hotel_df.insert(10, 'Hotel Name', " ")
hotel_df



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,County,Date,Hotel Name
96,96,bilma,18.69,12.92,75.42,19.0,0.0,1.5,NE,2020-10-19 23:33:13.877093,
145,145,santa isabel do rio negro,-0.41,-65.02,73.72,95.0,100.0,1.63,BR,2020-10-19 23:33:16.987484,
207,207,mandleshwar,22.18,75.67,75.2,83.0,31.0,1.77,IN,2020-10-19 23:33:20.880812,
353,353,fengxian,34.7,116.59,71.01,52.0,100.0,1.01,CN,2020-10-19 23:33:30.182207,
524,524,port macquarie,-31.43,152.92,71.01,56.0,29.0,1.99,AU,2020-10-19 23:33:41.396418,
531,531,nakanojo,36.58,138.85,71.01,45.0,16.0,1.01,JP,2020-10-19 23:33:41.959239,
540,540,riberalta,-10.98,-66.1,72.9,88.0,100.0,1.66,BO,2020-10-19 23:33:42.495068,


In [7]:
# create a params dict that will be updated with new city each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


c:\users\zach\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\zach\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Missing field/result... skipping.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,County,Date,Hotel Name
96,96,bilma,18.69,12.92,75.42,19.0,0.0,1.5,NE,2020-10-19 23:33:13.877093,
145,145,santa isabel do rio negro,-0.41,-65.02,73.72,95.0,100.0,1.63,BR,2020-10-19 23:33:16.987484,Hotel Ayuana
207,207,mandleshwar,22.18,75.67,75.2,83.0,31.0,1.77,IN,2020-10-19 23:33:20.880812,
353,353,fengxian,34.7,116.59,71.01,52.0,100.0,1.01,CN,2020-10-19 23:33:30.182207,GreenTree Inn JiangSu XuZhou Feng County LiuBa...
524,524,port macquarie,-31.43,152.92,71.01,56.0,29.0,1.99,AU,2020-10-19 23:33:41.396418,Sails Port Macquarie by Rydges
531,531,nakanojo,36.58,138.85,71.01,45.0,16.0,1.01,JP,2020-10-19 23:33:41.959239,Forest Resort Conifer Iwabitsu
540,540,riberalta,-10.98,-66.1,72.9,88.0,100.0,1.66,BO,2020-10-19 23:33:42.495068,Pacifico Apart Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{County}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [11]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…